### Saturday, December 2, 2023

https://huggingface.co/Intel/neural-chat-7b-v3-1

docker container start hfpt_Oct28

In [3]:
!ls /home/rob/Data2/huggingface/transformers

models--EleutherAI--gpt-neox-20b	     models--mistralai--Mistral-7B-v0.1
models--meta-llama--Llama-2-13b-chat-hf      models--mosaicml--mpt-7b-instruct
models--meta-llama--Llama-2-13b-hf	     tmp_35pwa_q
models--meta-llama--Llama-2-7b-chat-hf	     tmphxgsvifp
models--meta-llama--Llama-2-7b-hf	     tmpjiz9wrho
models--mistralai--Mistral-7B-Instruct-v0.1  version.txt


In [2]:
# This download did not complete, so let's whack what we have ...
# !rm -rf /home/rob/Data2/huggingface/transformers/models--Intel--neural-chat-7b-v3-1

In [1]:
import transformers

model_name = 'Intel/neural-chat-7b-v3-1'

In [2]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)

OSError: Consistency check failed: file should be of size 9942981496 but has size 1792180273 ((…)of-00002.safetensors).
We are sorry for the inconvenience. Please retry download and pass `force_download=True, resume_download=False` as argument.
If the issue persists, please let us know by opening an issue on https://github.com/huggingface/huggingface_hub.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [ ]:
def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, max_length=1000, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]

In [ ]:
# Example usage
system_input = "You are a math expert assistant. Your mission is to help users understand and solve various math problems. You should provide step-by-step solutions, explain reasonings and give the correct answer."
user_input = "calculate 100 + 520 + 60"
response = generate_response(system_input, user_input)
print(response)

In [ ]:
# expected response
"""
To calculate the sum of 100, 520, and 60, we will follow these steps:

1. Add the first two numbers: 100 + 520
2. Add the result from step 1 to the third number: (100 + 520) + 60

Step 1: Add 100 and 520
100 + 520 = 620

Step 2: Add the result from step 1 to the third number (60)
(620) + 60 = 680

So, the sum of 100, 520, and 60 is 680.
"""